# ❄️ Snowflake Cortex Search ❄️

- A performant, hybrid search engine for unstructured data
- It unlocks scalable, business-specific LLM apps in Snowflake
- Is a foundational building block for LLM applications in Snowflake


In [ ]:
#Import python packages & establish session
import pandas as pd
from PyPDF2 import PdfFileReader
from snowflake.snowpark.files import SnowflakeFile
from io import BytesIO
from snowflake.snowpark.types import StringType, StructField, StructType
from langchain.text_splitter import RecursiveCharacterTextSplitter

from snowflake.snowpark.context import get_active_session
session = get_active_session()


In [ ]:
ls @resumes

In [ ]:
# Create a Snowpark based function to extract text from PDFs
def readpf(file_path):
    whole_text = ""
    with SnowflakeFile.open(file_path, 'rb') as file:
        f = BytesIO(file.readall())
        pdf_reader = PdfFileReader(f)
        whole_text = ""
        for page in pdf_reader.pages:
            whole_text += page.extract_text()
    return whole_text


In [ ]:
# Register the UDF
# Convert the cell to markdown to prevent rerunning later
session.udf.register(
    func = readpf
  , return_type = StringType()
  , input_types = [StringType()]
  , is_permanent = True
  , name = 'SNOWPARK_PDF'
  , replace = True
  , packages=['snowflake-snowpark-python','pypdf2']
  , stage_location = '@resumes'
)


In [ ]:
CREATE OR REPLACE TABLE RESUMES_TEXT AS
SELECT 
    relative_path, 
    file_url, 
    SNOWPARK_PDF(build_scoped_file_url(@resumes, relative_path)) as RESUMES_TEXT
from directory(@resumes)
WHERE RELATIVE_PATH like '%pdf';


In [ ]:
SELECT * from RESUMES_TEXT;

In [ ]:
SELECT * from CLEAN_RESUMES_TEXT;

In [ ]:
-- Create Cortex Search Service on your Clean_Resumes_Text table
CREATE OR REPLACE CORTEX SEARCH SERVICE RESUME_SEARCH_SERVICE
  ON RESUMES_TEXT
  WAREHOUSE = DATASCIENCECOLLEGE
  TARGET_LAG = '30 days'
  AS (
    SELECT
        relative_path, 
        file_url,
        resumes_text
    FROM CLEAN_RESUMES_TEXT
);


In [ ]:
from snowflake.snowpark import Session
from snowflake.core import Root
root=Root(session)

transcript_search_service = (root
  .databases['DATASCIENCECOLLEGE']
  .schemas['PUBLIC']
  .cortex_search_services['RESUME_SEARCH_SERVICE']
)

resp = transcript_search_service.search(
  query='who is a lead web developer?',
  columns=['RESUMES_TEXT','file_url'],
  #filter={"@eq": {"<column>": "<value>"} },
  limit=3
)
#https://docs.snowflake.com/user-guide/snowflake-cortex/cortex-search/query-cortex-search-service
#Rows where the "array_col" column contains "arr_value" and the "string_col" column equals "value":
#{
#    "@and": [
#      { "@contains": { "array_col": "arr_value" } },
#      { "@eq": { "string_col": "value" } }
#    ]

print(resp.to_json())


In [ ]:
results = resp.results

context_str = ""
for i, r in enumerate(results):
    context_str += f"Context document {i+1}: {r['RESUMES_TEXT']}\n****************\n"

print(context_str)


In [ ]:
SHOW CORTEX SEARCH SERVICES STARTS WITH 'RESUME_SEARCH_SERVICE';


In [ ]:
select util_db.public.se_grader(step, (actual = expected), actual, expected, description) as graded_results from 
(SELECT 'SEAI20' as step
,( SELECT COUNT(*) FROM TABLE(RESULT_SCAN(LAST_QUERY_ID()))) as actual
 , 1 as expected
 ,'HOL DORA Evaluation Test #7 IS CORRECT!' as description);


In [ ]:
df = session.create_dataframe(resp.results)
df.create_or_replace_temp_view("temp_resumes")


In [ ]:
SELECT SNOWFLAKE.CORTEX.COMPLETE(
    'mistral-large',
        CONCAT(
            'You are hiring a lead web app developer. Which candidate has the most web app development experience? Critique the candidates in bullet points: <results>',
            (SELECT LISTAGG(RESUMES_TEXT, ' ') FROM temp_resumes),
            '</results>'
        )
    ) AS CRITIQUE
    FROM temp_resumes LIMIT 1;
